## Importing libraries

In [1]:
import numpy as np
import pandas as pd
import orca
from matplotlib import pyplot as plt
import matplotlib.animation as animation
import seaborn as sns
import os; os.chdir('../../')
import warnings; warnings.simplefilter('ignore')

import pandana as pdna
from collections import OrderedDict
import pylogit as pl
from pylogit import nested_logit as nl
import math 
import pickle
import dill
import time
import random
import scipy.stats as st
from sklearn.neighbors import BallTree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
    
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep
from urbansim_templates.models import BinaryLogitStep
from urbansim.utils import misc
import urbansim_templates

from scripts import datasources, models, variables, utils

Registering model step 'TOD_choice'
Registering model step 'auto_ownership'
Registering model step 'WLCM_gen_tt'
Registering model step 'primary_mode_choice'
Registering model step 'WLCM'


## Preparing data for simulation

In [2]:
input_file_format = 'csv'
input_data_dir = 's3://baus-data/spring_2019/'

formattable_fname_dict = {
    'parcels': 'parcels.{0}',
    'buildings': 'buildings.{0}',
    'jobs': 'jobs.{0}',
    'establishments': 'establishments.{0}',
    'households': 'households.{0}',
    'persons': 'persons.{0}',
    'rentals': 'craigslist.{0}',
    'units': 'units.{0}',
    'mtc_skims': 'mtc_skims.{0}',
    'beam_skims_raw': '30.skims-smart-23April2019-baseline.csv.gz',
    'beam_skims_imputed': 'beam_skims_imputed.{0}',
    # the following nodes and edges .csv's will be phased out and
    # replaced by travel model skims entirely
    'drive_nodes': 'drive_nodes.{0}',
    'drive_edges': 'drive_edges.{0}',
    'drive_access_vars': 'drive_net_vars.{0}',
    'walk_nodes': 'walk_nodes.{0}',
    'walk_edges': 'walk_edges.{0}',
    'walk_access_vars': 'walk_net_vars.{0}',
    'zones': 'zones.{0}',
    'zone_access_vars': 'zones_w_access_vars.{0}',
    'students_k12': 'students_k12.{0}',
    'schools_k12': 'schools_bay_area.{0}'
}

def format_fname_dict(formattable_fname_dict, format='csv'):
    formatted_dict = {
        k: v.format('csv')
        for k, v in formattable_fname_dict.items()}
    return formatted_dict

input_fnames = format_fname_dict(
            formattable_fname_dict, input_file_format)

In [3]:
orca.add_injectable('input_file_format', input_file_format)
orca.add_injectable('input_data_dir', input_data_dir)
orca.add_injectable('input_fnames', input_fnames)
orca.add_injectable('store', None)

In [4]:
orca.run(['initialize_network_small']) 
orca.run(['initialize_network_walk']) 
orca.run(['initialize_imputed_skims']) 
orca.run(['skims_aggregations']) 

Running step 'initialize_network_small'
Time to execute step 'initialize_network_small': 0.00 s
Total time to execute iteration 1 with iteration value None: 0.00 s
Running step 'initialize_network_walk'
Time to execute step 'initialize_network_walk': 0.00 s
Total time to execute iteration 1 with iteration value None: 0.00 s
Running step 'initialize_imputed_skims'
No imputed skims found. Creating them now.
Time to execute step 'initialize_imputed_skims': 118.41 s
Total time to execute iteration 1 with iteration value None: 118.41 s
Running step 'skims_aggregations'
Time to execute step 'skims_aggregations': 0.00 s
Total time to execute iteration 1 with iteration value None: 0.00 s


## preparing data for simulation

In [12]:
pu = orca.get_table('public_schools_50').to_frame()
persons = orca.get_table('persons').to_frame()

In [14]:
persons.columns

Index(['member_id', 'age', 'primary_commute_mode', 'relate', 'edu', 'sex',
       'hours', 'hispanic', 'earning', 'race_id', 'student', 'work_at_home',
       'worker', 'household_id', 'node_id_small', 'node_id_walk',
       'no_higher_ed', 'age_under_45', 'zone_id_home', 'lessGED', 'GED',
       'somebach', 'Assoc', 'Bach', 'female', 'white', 'minority',
       'age_16less25', 'age_0_5', 'age_5_12', 'age_12_15', 'age_15_18',
       'black', 'asian', 'sex_SMC', 'race_white', 'race_african_american',
       'race_indian/alaska', 'race_asian', 'race_hawaii/pacific', 'race_other',
       'race_2+races', 'children', 'age_0_15', 'age_16_17', 'age_18_25',
       'age_26_40', 'age_41_60', 'age_60', 'worker_student',
       'non_worker_non_student'],
      dtype='object')

In [5]:
nodessmall = pd.read_csv('/home/ubuntu/activitysynth/activitysynth/data/nodessmall.csv').set_index('osmid')
nodeswalk = pd.read_csv('/home/ubuntu/activitysynth/activitysynth/data/nodeswalk.csv').set_index('osmid')

orca.add_table('nodessmall', nodessmall)

In [6]:
orca.broadcast(cast = 'public_schools_50', onto = 'persons', cast_index= True, onto_on = 'node_id_small')

In [7]:
merge = orca.merge_tables(target='persons', tables=['persons','public_schools_50'], )
#                           columns = ['mean5'])

MemoryError: 

In [ ]:
merge

## Simulation 

In [5]:
@orca.step()
def SLCM_simulate(persons):
    """
    Generate school location choices for the synthetic pop households.
    
    """
    #Running the simulation 
    file_Name = "/home/juan/activitysynth/activitysynth/configs/SLCM_gen_tt.pkl"
    fileObject = open(file_Name,'rb')  
    SLCM = pickle.load(fileObject) 
    orca.add_column('long_format', 'probabilities', SLCM.predict(orca.get_table('long_format').to_frame()))
    df = orca.get_table('long_format').to_frame()
    school_choice = df.sort_values("probabilities", ascending=False).groupby('obs_id').agg({'school_choice_set': 'first'})
    
    #Merging schools in the persons table 
    persons_with_school = persons.to_frame().merge(school_choice, 
                                                   how = 'left', 
                                                   left_index = True, 
                                                   right_index = True).rename({'school_choice_set': 'school_id'})
    
    orca.add_table('persons', persons_with_school)
    

In [6]:
orca.run(['SLCM_simulate']) 

Running step 'SLCM_simulate'


FileNotFoundError: [Errno 2] No such file or directory: '/home/juan/activitysynth/activitysynth/configs/SLCM_gen_tt.pkl'

In [9]:
persons = orca.get_table('persons').to_frame()